In [2]:
import redis
import os
import numpy as np
import pandas as pd

r = redis.Redis(host='localhost', port=6379)

input_url = "bosch_cnc"
metadata_url = os.path.join(
    input_url,
    "metadata.csv"
)
metadata_df = pd.read_csv(metadata_url)
metadata_df.head()

,filename,time_frame,machine_id,op_id,tool_id,sensor_id,sampling_f,max,dtype,status
0,0.csv,1,0,3,0,0,2000,8192,<i2,bad
1,1.csv,2,0,3,0,0,2000,8192,<i2,good
2,2.csv,1,0,3,0,0,2000,8192,<i2,good
3,3.csv,0,0,3,0,0,2000,8192,<i2,good
4,4.csv,2,0,3,0,0,2000,8192,<i2,good


## Load Signal Data into Redis

In [52]:
DATA_COLUMNS = {
    "x": np.float32,
    "y": np.float32,
    "z": np.float32,
}
filenames = os.listdir(input_url)
filenames.remove('metadata.csv')
filenames.remove('.DS_Store')

for filename in filenames:
    file_path = os.path.join(input_url, filename)
    data_df = pd.read_csv(
        file_path, 
        names=DATA_COLUMNS.keys(),
        dtype=DATA_COLUMNS,
    )
    
    
    x = np.array(data_df['x']).tobytes()
    y = np.array(data_df['y']).tobytes()
    z = np.array(data_df['z']).tobytes()
    mapping = {
        'x': x,
        'y': y,
        'z': z
    }
    signal_name = filename.split('.')[0]

    r.hset(name='signal:'+signal_name, mapping=mapping)
    
    


In [53]:
FILES = []
for file in r.keys(pattern='signal:*'):
    FILES.append(int(file.decode('utf-8').split(':')[-1]))
FILES.sort()
print(len(FILES))

1702


In [49]:
import torch
signal_name = FILES[0]
response = r.hget(name="signal:"+str(signal_name), key='x')

file_path = os.path.join(input_url, '0.csv')
data_df = pd.read_csv(
    file_path, 
    names=DATA_COLUMNS.keys(),
    dtype=DATA_COLUMNS,
)


signal = np.array(data_df['x'])

'''x_axis = np.frombuffer(r.hget(name="signal:"+str(filename), key='x'), dtype=np.dtype(float))
y_axis = np.frombuffer(r.hget(name="signal:"+str(filename), key='y'), dtype=np.dtype(float))
z_axis = np.frombuffer(r.hget(name="signal:"+str(filename), key='z'), dtype=np.dtype(float))

signal_1 = {
    'x':x_axis,
    'y':y_axis
}'''
#print(np.frombuffer(signal), dtype=np.dtype(float))

#print(np.frombuffer(response))
x = signal.tobytes(order='C')
conv = np.frombuffer(response, dtype=np.float32)
print(torch.fft.rfft(torch.Tensor(signal)).abs())
print(torch.fft.rfft(torch.Tensor(conv)).abs())

tensor([31169.9922, 51175.6172, 58363.3281,  ..., 29700.3594, 47237.5664,
        11392.8174])
tensor([31169.9922, 51175.6172, 58363.3281,  ..., 29700.3594, 47237.5664,
        11392.8174])


In [55]:
processed_signals = [0 for i in np.arange(len(FILES))]
print(len(processed_signals))

1702


## Load Metadata information into Redis

In [3]:
metadata_df.head()

,filename,time_frame,machine_id,op_id,tool_id,sensor_id,sampling_f,max,dtype,status
0,0.csv,1,0,3,0,0,2000,8192,<i2,bad
1,1.csv,2,0,3,0,0,2000,8192,<i2,good
2,2.csv,1,0,3,0,0,2000,8192,<i2,good
3,3.csv,0,0,3,0,0,2000,8192,<i2,good
4,4.csv,2,0,3,0,0,2000,8192,<i2,good


In [6]:
FILES = []
for file in r.keys(pattern='signal:*'):
    FILES.append(int(file.decode('utf-8').split(':')[-1]))
FILES.sort()
print(len(FILES))
print(FILES[:10])

1702
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
0.csv


In [17]:
for file in FILES:
    filename = str(file)+'.csv'
    metadata_row = metadata_df.loc[metadata_df['filename']==filename]
    mapping = {
        'machine_id' : str(metadata_row.machine_id.values[0]),
        'process_id' : str(metadata_row.op_id.values[0]),
        'sampling_f' : str(metadata_row.sampling_f.values[0]),
        'status'     : str(metadata_row.status.values[0]),
    }
    r.hset(name='signal:'+str(file), mapping=mapping)
